# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,labuan bajo,-8.4964,119.8877,296.50,86,11,2.88,ID,1720127123
1,1,waitangi,-43.9535,-176.5597,281.94,96,100,3.13,NZ,1720127124
2,2,benguela,-12.5763,13.4055,293.11,87,80,1.85,AO,1720126833
3,3,georgetown,5.4112,100.3354,298.12,84,20,0.51,MY,1720127126
4,4,port-aux-francais,-49.3500,70.2167,275.23,96,100,13.04,TF,1720127128


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 0.85,
    color = 'City',
    alpha=0.8,)

# Display the map
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 22) &
    (city_data_df['Humidity'] < 50) &
    (city_data_df['Cloudiness'] == 0)]


# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,lagoa formosa,-18.7786,-46.4075,294.27,41,0,2.20,BR,1720127140
45,45,hassi messaoud,31.6804,6.0729,310.07,14,0,2.57,DZ,1720127177
98,98,tindouf,27.6711,-8.1474,304.38,19,0,6.48,DZ,1720127243
105,105,wenatchee,47.4235,-120.3103,306.55,23,0,2.24,US,1720127251
113,113,north bend,43.4065,-124.2243,306.44,49,0,9.26,US,1720127261


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
14,lagoa formosa,BR,-18.7786,-46.4075,41,
45,hassi messaoud,DZ,31.6804,6.0729,14,
98,tindouf,DZ,27.6711,-8.1474,19,
105,wenatchee,US,47.4235,-120.3103,23,
113,north bend,US,43.4065,-124.2243,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation",
    "apiKey": geoapify_key,
    "limit":10}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


# Display sample data
hotel_df.head()

Starting hotel search
lagoa formosa - nearest hotel: Hotel HS
hassi messaoud - nearest hotel: AMC
tindouf - nearest hotel: محمد بوسبي
wenatchee - nearest hotel: Hilton Garden Inn
north bend - nearest hotel: Quality Inn & Suites at Coos Bay
zhangatas - nearest hotel: Арман
ulysses - nearest hotel: Single Tree Inn Hotel O
gumdag - nearest hotel: Mammettaganow Arazdurdy's House
altamont - nearest hotel: Quality Inn
nalut - nearest hotel: No hotel found
ghanzi - nearest hotel: Ghanzi farmhouse
balcarce - nearest hotel: 646 Hotel Balcarce
shahrud - nearest hotel: خانه معلم شاهرود
valparaiso - nearest hotel: Ibis Hotel
alcaniz - nearest hotel: Hotel Guadalope
fallon - nearest hotel: Econo Lodge
portland - nearest hotel: the Hoxton
vernon - nearest hotel: Polson Park Motel
dzhalagash - nearest hotel: No hotel found
siwa oasis - nearest hotel: Flower of the city hotel
shush - nearest hotel: هتل آپادانا
ha'il - nearest hotel: فندق كلاسيك
illizi - nearest hotel: بوناقة
el hadjira - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
14,lagoa formosa,BR,-18.7786,-46.4075,41,Hotel HS
45,hassi messaoud,DZ,31.6804,6.0729,14,AMC
98,tindouf,DZ,27.6711,-8.1474,19,محمد بوسبي
105,wenatchee,US,47.4235,-120.3103,23,Hilton Garden Inn
113,north bend,US,43.4065,-124.2243,49,Quality Inn & Suites at Coos Bay


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 1.5,
    color = 'City',
    alpha=0.8,
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'])

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)